In [1]:
import numpy as np
from collections import deque
import numpy as np
import time
import pandas as pd

In [2]:
def matrix_to_tuple(matrix):
    return (matrix[0][1], matrix[0][2],
            matrix[1][2])

def tuple_to_matrix(my_tuple):
    return np.array([[my_tuple[0], my_tuple[1], my_tuple[2]], 
                     [my_tuple[3], my_tuple[4], my_tuple[5]], 
                     [my_tuple[6], my_tuple[7], my_tuple[8]]])

def apply_mat(mat, index):
    if index==0:
        return mat @ A
    elif index==1:
        return mat @ B
    elif index==2:
        return mat @ C
    elif index==3:
        return mat @ D
    assert(1==2)

# B is the inverse of A
A = np.array([[1, 1, 0], [0, 1, 0], [0, 0, 1]])
B = np.array([[1, -1, 0], [0, 1, 0], [0, 0, 1]])

# C is the inverse of D
C = np.array([[1, 0, 0], [0, 1, 1], [0, 0, 1]])
D = np.array([[1, 0, 0], [0, 1, -1], [0, 0, 1]])

# together, A, B, C, and D generate the heisenberg group

I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [3]:
data = pd.DataFrame({
        'val1': [],
        'val2': [],
        'val3': [],
        'last_matrix': [],
        "num_steps": []
    })

I_as_tuple = matrix_to_tuple(I)

cur_row_df = pd.DataFrame([[
    I_as_tuple[0], I_as_tuple[1], I_as_tuple[2], 
    0, 0
    ]], columns=['val1', 'val2', 'val3', 'last_matrix', 'num_steps'])
data = pd.concat([cur_row_df, data])

In [4]:
data

,val1,val2,val3,last_matrix,num_steps
0,0.0,0.0,0.0,0.0,0.0


In [7]:
v = dict()

v[(1, 2, 3)] = [1, [1, 2]]
v
(1, 2, 3) in v

True

In [60]:
# adapted from ChatGPT
# visited contains keys of the tuples we've already visited and values of either 0 when A was the matrix that brought us to this state
# of 1 if B was the matrix that brought us to this state

visited = dict()

start_time = time.time()

def bfs(start, myDF):
    visited[(matrix_to_tuple(start))] = [0, []]
    # visited[(matrix_to_tuple(start @ A))] = [1, [0]]
    # visited[(matrix_to_tuple(start @ B))] = [1, [1]]
    # visited[(matrix_to_tuple(start @ C))] = [1, [2]]
    # visited[(matrix_to_tuple(start @ D))] = [1, [3]]
    
    queue = deque([[A, 1, 0], [B, 1, 1], [C, 1, 2], [D, 1, 3]])

    while queue:
        # print(queue)
        cur_item = queue.popleft()
        cur_mat = cur_item[0]
        cur_index = cur_item[1]
        cur_neighbor = cur_item[2]
        cur_tuple = matrix_to_tuple(cur_mat)

        # if time.time() >= start_time + 30:
        #     print("Timed out")
        #     return myDF
        if cur_index > 26:
            print("Got enough points")
            return myDF

        if cur_tuple in visited:
            if cur_index<visited[cur_tuple][0]:
                assert 1==2
            elif cur_index==visited[cur_tuple][0]:
                visited[cur_tuple][1].append(cur_neighbor)
                cur_row_df = pd.DataFrame([[
                    cur_tuple[0], cur_tuple[1], cur_tuple[2], 
                    cur_index, cur_neighbor
                    ]], columns=['val1', 'val2', 'val3', 'num_steps', 'last_matrix'])
                myDF = pd.concat([cur_row_df, myDF])
                continue
            # else is the case where the path is longer, so we don't care about it
            else: 
                continue

        cur_row_df = pd.DataFrame([[
            cur_tuple[0], cur_tuple[1], cur_tuple[2], 
            cur_index, cur_neighbor
            ]], columns=['val1', 'val2', 'val3', 'num_steps', 'last_matrix'])
        myDF = pd.concat([cur_row_df, myDF])

        visited[cur_tuple]=[cur_index, [cur_neighbor]]

        for neighbor in [0, 1, 2, 3]:
            new_mat = apply_mat(cur_mat, neighbor)
            queue.append([new_mat, cur_index+1, neighbor])
        

data = bfs(I, data)

Got enough points


In [61]:
data=data.reset_index(drop=True)
data

,val1,val2,val3,num_steps,last_matrix
0,0,0,-26,26,3
1,-1,0,-25,26,1
2,1,0,-25,26,0
3,-1,1,-25,26,3
4,-1,-1,-23,26,2
...,...,...,...,...,...
409793,2,0,0,2,0
409794,0,0,-1,1,3
409795,0,0,1,1,2
409796,-1,0,0,1,1


In [62]:
few_rows = data[(data['val1']==-14) & (data['val2']==-26) & (data['val3']==2)]
few_rows

,val1,val2,val3,num_steps,last_matrix
316947,-14,-26,2,16,2
316958,-14,-26,2,16,1
367529,-14,-26,2,16,2
367540,-14,-26,2,16,1


In [25]:
testing = np.array([[1, -14, -26], [0, 1, 2], [0, 0, 1]])
testing @ A


array([[  1, -13, -26],
       [  0,   1,   2],
       [  0,   0,   1]])

In [58]:
visited[matrix_to_tuple(testing @ D @ A @ A @ D @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A)]

[0, []]

In [41]:
visited[matrix_to_tuple(testing @ A @ D @ D @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A)]

[0, []]

In [59]:
testing @ A @ D @ D @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A
testing @ D @ A @ A @ D @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A @ A

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [21]:
grouped = data.groupby(['val1', 'val2', 'val3']).mean()
grouped[grouped['last_matrix'] % 1 != 0]

num_steps  last_matrix
val1 val2 val3                        
-14  -26  2          16.0          1.5
     -25  2          16.0          1.5
     -24  2          16.0          1.5
     -23  2          16.0          1.5
     -22  2          16.0          1.5
...                   ...          ...
 15  -5   0          17.0          1.5
     -4   0          17.0          1.5
     -3   0          17.0          1.5
     -2   0          17.0          1.5
     -1   0          17.0          1.5

[9433 rows x 2 columns]

In [65]:
print(grouped.shape)
print(data.drop_duplicates().shape)
data.shape

(28975, 2)
(359216, 5)


(409798, 5)

In [63]:
no_dups = data.drop_duplicates()

In [66]:
# data = data.drop_duplicates(subset=['val1', 'val2', 'val3'])
no_dups.to_csv("../Data_Generation/Data_files/bfs_heisenberg_w_mult_paths.csv", index=False)

In [94]:
duplicate_rows = data[data.duplicated(keep=False)]
duplicate_rows

,val1,val2,val3,num_steps,last_matrix
